In [4]:
import numpy as np
import panel as pn
import pandas as pd
from statsmodels.regression.linear_model import OLSResults
from typing import Dict
import param
pn.extension()

In [2]:
# load model
model = OLSResults.load("model.pickle")
zooms = range(6)

In [73]:
def predict_tile_times(
       lat_dim_size: int,
       lon_dim_size: int,
       lat_chunk_size: int,
       lon_chunk_size: int,
       time_chunk_size: int,
       data_type: str
   ) -> Dict[str, float]:
    number_of_spatial_chunks = (lat_dim_size/lat_chunk_size) * (lon_dim_size/lon_chunk_size)
    data_dtype = np.dtype(data_type)
    chunk_size_mb = lat_chunk_size * lon_chunk_size * time_chunk_size * data_dtype.itemsize/1024/1024
    print(f"Chunk Size: {chunk_size_mb} MB")
    values_to_predict = {
        'const': [1],
        'number_of_spatial_chunks': [number_of_spatial_chunks],
        'chunk_size_mb': [chunk_size_mb]
    }
    tile_times = {}
    for zoom in zooms:
        zooms_to_predict = {f'zoom {zoom}': [0] for zoom in zooms}        
        zooms_to_predict[f'zoom {zoom}'] = [1]
        values_to_predict.update(zooms_to_predict)
        prediction = model.predict(pd.DataFrame(values_to_predict))
        tile_times[f'zoom {zoom}'] = prediction
    return tile_times

In [74]:
class App(param.Parameterized):
    lat_dim_size_widget = param.Integer(600, label='Latitidude Dimension Size')
    lon_dim_size_widget = param.Integer(1440, label='Longitidude Dimension Size')

    lat_chunk_size_widget = param.Integer(600, label='Latitidude Chunk Size')
    lon_chunk_size_widget = param.Integer(1440, label='Longitidude Chunk Size')
    time_chunk_size_widget = param.Integer(1, label='Time Chunk Size')

    data_type_widget = param.Selector(["float16", "float32", "float64"], label="Data Type")
    
    @param.depends('lat_dim_size_widget', 'lon_dim_size_widget', 'lat_chunk_size_widget', 'lon_chunk_size_widget', 'time_chunk_size_widget', 'data_type_widget', watch=False)
    def compute_output(self):
        # Some computation based on the sliders
        result = predict_tile_times(
            lat_dim_size=self.lat_dim_size_widget,
            lon_dim_size=self.lon_dim_size_widget,
            lat_chunk_size=self.lat_chunk_size_widget,
            lon_chunk_size=self.lon_chunk_size_widget,
            time_chunk_size=self.time_chunk_size_widget,
            data_type=self.data_type_widget,
        )
        return result

In [76]:
app = App()

# Create a Panel layout
layout = pn.Column(app.param, app.compute_output)

# Serve the app
layout.servable()

Chunk Size: 1.64794921875 MB


Chunk Size: 3.2958984375 MB

Column
    [0] Column(margin=(5, 10), name='App')
        [0] StaticText(value='<b>App</b>')
        [1] IntInput(name='Latitidude Dimension S..., value=600)
        [2] IntInput(name='Longitidude D..., value=1440)
        [3] IntInput(name='Latitidude Chunk Size', value=600)
        [4] IntInput(name='Longitidude Chunk Size', value=1440)
        [5] IntInput(name='Time Chunk Size', value=1)
        [6] Select(name='Data Type', options=OrderedDict([('float16', ...]), value='float16')
    [1] ParamMethod(method, _pane=Str, defer_load=False)